In [2]:
from datetime import datetime
import numpy as np
import pandas as pd

In [16]:
data = pd.read_csv("data/bitmex/20220123.csv")
data = data.append(pd.read_csv("data/bitmex/20220124.csv"))
data = data.append(pd.read_csv("data/bitmex/20220125.csv"))

In [17]:
# vwap
def compute_vwap(df):
    p, q = data['price'], data['foreignNotional']
    df['vwap'] = np.sum(p * q) / np.sum(q)
    return df

# ohlv 
def compute_ohlv(df):
    df['open'] = df.price.iloc[0]
    df['close'] = df.price.iloc[-1]
    df['low'] = df.price.min()
    df['high'] = df.price.max()
    return df[-1:]

data = data[data.symbol == 'XBTUSD']






In [18]:
data

,timestamp,symbol,side,size,price,tickDirection,trdMatchID,grossValue,homeNotional,foreignNotional
121973,2022-01-23D00:00:00.271578000,XBTUSD,Buy,2900,35052.0,ZeroPlusTick,a0aa992a-5c15-a099-9d03-a71aee142835,8273410,0.082734,2900.0
121974,2022-01-23D00:00:00.271578000,XBTUSD,Buy,1900,35052.0,ZeroPlusTick,5e9f715c-1f87-5598-75df-027facf7c08d,5420510,0.054205,1900.0
121975,2022-01-23D00:00:00.271578000,XBTUSD,Buy,5000,35052.0,ZeroPlusTick,1fab656b-b637-632c-e05b-30214d05bab5,14264500,0.142645,5000.0
121976,2022-01-23D00:00:11.543174000,XBTUSD,Sell,1000,35061.0,PlusTick,0a5518ba-8365-99eb-4ba2-ba1a5155311b,2852170,0.028522,1000.0
121977,2022-01-23D00:00:15.534975000,XBTUSD,Buy,100,35061.5,PlusTick,ec28ea49-ac7a-aa29-fd68-6ce0c11ecce3,285213,0.002852,100.0
...,...,...,...,...,...,...,...,...,...,...
276852,2022-01-25D23:59:56.454606000,XBTUSD,Buy,7400,36941.5,ZeroPlusTick,25448eb7-49a8-e482-879f-0f5768f2fd6e,20031652,0.200317,7400.0
276853,2022-01-25D23:59:56.454606000,XBTUSD,Buy,7000,36941.5,ZeroPlusTick,505ce106-94fb-e5a0-9a2f-ced139414048,18948860,0.189489,7000.0
276854,2022-01-25D23:59:56.454606000,XBTUSD,Buy,5200,36944.0,PlusTick,e3049a18-daaa-ecdf-7701-aca51d2edc31,14075360,0.140754,5200.0
276855,2022-01-25D23:59:59.035716000,XBTUSD,Sell,100,36949.0,PlusTick,3ede3f0b-0e90-b765-6a15-67aeb1d49fb1,270643,0.002706,100.0


In [19]:
data['timestamp'] = data.timestamp.map(lambda t: datetime.strptime(t[:-3], "%Y-%m-%dD%H:%M:%S.%f"))
data.set_index('timestamp', inplace=True)


In [20]:
data.sort_index(inplace=True)

In [22]:
# create dollar bars
data_cm_dollar = data.assign(cmDollar = data['foreignNotional'].cumsum())
total_dollars = data_cm_dollar.cmDollar.values[-1]
num_dollars_per_bar = 2e6


In [35]:
data_dollar_group = data_cm_dollar.assign(grpId = lambda row: row.cmDollar // num_dollars_per_bar)
data_dollar_ohlc = data_dollar_group.groupby('grpId').apply(lambda x: compute_ohlv(x))


In [36]:
data_dollar_ohlc.index = data_dollar_ohlc.index.droplevel()

In [37]:
data_dollar_ohlc.index

DatetimeIndex(['2022-01-23 00:06:54.844749', '2022-01-23 00:08:42.310086',
               '2022-01-23 00:10:39.370645', '2022-01-23 00:15:26.089176',
               '2022-01-23 00:22:39.768537', '2022-01-23 00:27:27.580947',
               '2022-01-23 00:35:56.558531', '2022-01-23 00:42:02.961714',
               '2022-01-23 00:48:42.122213', '2022-01-23 00:57:21.181146',
               ...
               '2022-01-25 22:51:01.505483', '2022-01-25 23:01:38.541974',
               '2022-01-25 23:08:47.130731', '2022-01-25 23:15:18.402171',
               '2022-01-25 23:20:42.999844', '2022-01-25 23:30:58.413313',
               '2022-01-25 23:37:17.317940', '2022-01-25 23:47:38.415232',
               '2022-01-25 23:55:46.511007', '2022-01-25 23:59:59.466370'],
              dtype='datetime64[ns]', name='timestamp', length=1709, freq=None)

In [38]:
data_dollar_ohlc = data_dollar_ohlc[~data_dollar_ohlc.index.duplicated(keep='first')]

In [42]:
def get_volatility(prices, span = 100, delta = pd.Timedelta(hours=1)):
    df0 = prices.index.searchsorted(prices.index, prices.index - delta)
    df0 = df0[df0 > 0]
    return df0


new = get_volatility(data_dollar_ohlc)
new


TypeError: value should be a 'Timestamp', 'NaT', or array of those. Got 'str' instead.